# Use Uhunt API

In [10]:
import requests
import json
import pandas as pd

class UhuntAPI:
    def __init__(self):
        self.base_url = 'https://uhunt.onlinejudge.org/api'
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'Mozilla/5.0'})
    
    def get_problems(self):
        path = '/p'
        url = f'{self.base_url}{path}'
        response = self.session.get(url)
        if response.status_code == 200:
            data = response.json()
            columns = ['pid', 'num', 'title', 'dacu', 'mrun', 'mmem', 'nover', 'sube', 'noj', 'inq', 'ce', 'rf', 're', 'ole', 'tle', 'mle', 'wa', 'pe', 'ac', 'rtl', 'status', 'rej']
            return pd.DataFrame(data, columns=columns)
        else:
            raise Exception(f"Failed to fetch problems: {response.status_code}")

    def get_problem_submissions(self, pid, start_sbt=0, end_sbt=2147483647):
        path = f'/p/subs/{pid}/{start_sbt}/{end_sbt}'
        url = f'{self.base_url}{path}'
        response = self.session.get(url)
        if response.status_code == 200:
            data = response.json()
            columns = ['sid', 'uid', 'pid', 'ver', 'lan', 'run', 'mem', 'rank', 'sbt', 'name', 'uname']
            return pd.DataFrame(data, columns=columns)
        else:
            raise Exception(f"Failed to fetch submissions for problem {pid}: {response.status_code}")


# Get Data

In [ ]:
import os

if __name__ == '__main__':
    try:
        api = UhuntAPI()
        problems_df = api.get_problems()
        submissions_dir = 'submissions'
        if not os.path.exists(submissions_dir):
            os.makedirs(submissions_dir)
        for index, row in problems_df.iterrows():
            path = os.path.join(submissions_dir, f'{row["num"]}.csv')
            if os.path.exists(path):
                prev_submissions_df = pd.read_csv(path)
                last_submission_time = prev_submissions_df['sbt'].max()
                submissions_df = api.get_problem_submissions(row['pid'], last_submission_time + 1)
                submissions_df = pd.concat([prev_submissions_df, submissions_df], ignore_index=True)
                submissions_df.drop_duplicates(subset=['sid'], keep='last', inplace=True)
            else:
                submissions_df = api.get_problem_submissions(row['pid'])
            submissions_df.to_csv(path, index=False)
    except Exception as e:
        print(e)

[Errno 22] Invalid argument: 'submissions\\127 "Accordian" Patience.csv'
